In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-logicalcombinations/logical_combinations_output.json


In [2]:
import pandas as pd
import json

# Load JSON
with open("/kaggle/input/csqa-logicalcombinations/logical_combinations_output.json", "r") as f:
    data = json.load(f)

# Convert the 'questions' list to a DataFrame
df = pd.DataFrame(data["questions"])
df.to_csv('logical_combinations_output.csv',index=False)

df.head()


,question,original_correct_facts,original_incorrect_facts,logical_combinations
0,The sanctions against the school were a punish...,"[undermine, invalidate, negate]","[enforce rules, promote discipline, facilitate...",{'AND_combinations': {'correct': ['undermine A...
1,Sammy wanted to go to where the people were. ...,"[community gatherings, local events, social ve...","[sports arenas, quiet retreats, train platform...",{'AND_combinations': {'correct': ['community g...
2,To locate a choker not located in a jewelry bo...,"[artisan market, vintage fair, online marketpl...","[fashion outlet, home decor store, personal ac...",{'AND_combinations': {'correct': ['artisan mar...
3,Google Maps and other highway and street GPS s...,"[printed road maps, manual navigation techniqu...","[local traffic reports, historical navigation ...",{'AND_combinations': {'correct': ['printed roa...
4,"The fox walked from the city into the forest, ...","[natural habitat, suitable prey, shelter from ...","[urban garden, city park, farm fields, neighbo...",{'AND_combinations': {'correct': ['natural hab...


In [3]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [5]:
from huggingface_hub import login
login(new_session=False)

In [6]:
import pandas as pd
import random
import ast

# Load your dataset
df = pd.read_csv("/kaggle/working/logical_combinations_output.csv")  # replace with your file

# Safe eval to ensure lists are properly loaded
def safe_eval(val):
    if isinstance(val, list):
        return val
    if pd.isna(val):
        return []
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return []
    return []

df['logical_combinations'] = df['logical_combinations'].apply(safe_eval)

# Prepare storage for QA sets
qa_sets = {
    'AND': [],
    'OR': [],
    'NEITHER': [],
    'Mixed': []
}

# Label rotation - separate rotator for each question type
class LabelRotator:
    def __init__(self, name):
        self.name = name
        self.labels = ['A', 'B', 'C', 'D']
        self.current_index = 0
    
    def get_next_label(self):
        label = self.labels[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.labels)
        # print(f"DEBUG {self.name}: Assigned label {label}, next will be {self.labels[self.current_index]}")
        return label

# Create separate rotators for each question type
rotators = {
    'AND': LabelRotator('AND'),
    'OR': LabelRotator('OR'),
    'NEITHER': LabelRotator('NEITHER'),
    'Mixed': LabelRotator('Mixed')
}

def create_qa_item(question, correct_ans, incorrect_ans, qa_type):
    """Helper function to create a QA item with proper label rotation"""
    
    # Get correct label from the appropriate rotator
    correct_label = rotators[qa_type].get_next_label()
    
    # Ensure we have exactly 3 incorrect answers
    if len(incorrect_ans) < 3:
        print(f"Warning: Only {len(incorrect_ans)} incorrect answers available for {qa_type}")
        return None
    
    selected_incorrect = random.sample(incorrect_ans, 3)
    
    # Create options array with 4 positions
    options = [''] * 4
    
    # Place correct answer at the designated position
    correct_pos = rotators[qa_type].labels.index(correct_label)
    options[correct_pos] = correct_ans
    
    # Fill remaining positions with incorrect answers
    incorrect_positions = [i for i in range(4) if i != correct_pos]
    for i, pos in enumerate(incorrect_positions):
        options[pos] = selected_incorrect[i]
    
    return {
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans,
        'qa_type': qa_type
    }

# Track counts for each type
type_counts = {'AND': 0, 'OR': 0, 'NEITHER': 0, 'Mixed': 0}

for idx, row in df.iterrows():
    question = row['question']
    combos = row['logical_combinations']
    
    # Extract AND/OR/NEITHER correct and incorrect lists
    and_correct = combos.get('AND_combinations', {}).get('correct', [])
    and_incorrect = combos.get('AND_combinations', {}).get('incorrect', [])
    
    or_correct = combos.get('OR_combinations', {}).get('correct', [])
    or_incorrect = combos.get('OR_combinations', {}).get('incorrect', [])
    
    neither_correct = combos.get('NEITHER_combinations', {}).get('correct', [])
    neither_incorrect = combos.get('NEITHER_combinations', {}).get('incorrect', [])
    
    # --- AND-only QA ---
    if and_correct and len(and_incorrect) >= 3:
        correct_ans = random.choice(and_correct)
        qa_item = create_qa_item(question, correct_ans, and_incorrect, 'AND')
        if qa_item:
            qa_sets['AND'].append(qa_item)
            type_counts['AND'] += 1
            # print(f"AND Question {type_counts['AND']}: Label {qa_item['correct_label']}")
    
    # --- OR-only QA ---
    if or_correct and len(or_incorrect) >= 3:
        correct_ans = random.choice(or_correct)
        qa_item = create_qa_item(question, correct_ans, or_incorrect, 'OR')
        if qa_item:
            qa_sets['OR'].append(qa_item)
            type_counts['OR'] += 1
            # print(f"OR Question {type_counts['OR']}: Label {qa_item['correct_label']}")
    
    # --- NEITHER-only QA ---
    if neither_correct and len(neither_incorrect) >= 3:
        correct_ans = random.choice(neither_correct)
        qa_item = create_qa_item(question, correct_ans, neither_incorrect, 'NEITHER')
        if qa_item:
            qa_sets['NEITHER'].append(qa_item)
            type_counts['NEITHER'] += 1
            # print(f"NEITHER Question {type_counts['NEITHER']}: Label {qa_item['correct_label']}")
    
    # --- Mixed QA ---
    all_correct_types = [
        ('AND', and_correct),
        ('OR', or_correct),
        ('NEITHER', neither_correct)
    ]
    all_incorrect_types = and_incorrect + or_incorrect + neither_incorrect
    
    # Ensure at least one correct exists and enough incorrect answers
    valid_categories = [cat for cat, lst in all_correct_types if lst]
    if valid_categories and len(all_incorrect_types) >= 3:
        chosen_category = random.choice(valid_categories)
        chosen_correct_list = dict(all_correct_types)[chosen_category]
        correct_ans = random.choice(chosen_correct_list)
        qa_item = create_qa_item(question, correct_ans, all_incorrect_types, 'Mixed')
        if qa_item:
            qa_sets['Mixed'].append(qa_item)
            type_counts['Mixed'] += 1
            # print(f"Mixed Question {type_counts['Mixed']}: Label {qa_item['correct_label']}")

print(f"\n=== GENERATION COMPLETE ===")
for qtype, count in type_counts.items():
    print(f"{qtype}: {count} questions generated")

# --- Convert to DataFrames and save ---
for key, qlist in qa_sets.items():
    if qlist:  # Only save if we have questions
        df_out = pd.DataFrame(qlist)
        df_out.to_csv(f'QA_{key}.csv', index=False)
        print(f"\n{key} QA saved: {len(df_out)} questions in QA_{key}.csv")
        
        # Show label distribution for verification
        label_counts = df_out['correct_label'].value_counts().sort_index()
        print(f"  Label distribution: {dict(label_counts)}")
        
        # Show label sequence for first 8 questions to verify rotation
        print(f"  Label sequence (first 8): {list(df_out['correct_label'].head(8))}")
    else:
        print(f"{key}: No valid QA pairs generated")

# Overall statistics
total_questions = sum(len(qlist) for qlist in qa_sets.values())
print(f"\nFinal total questions generated: {total_questions}")


=== GENERATION COMPLETE ===
AND: 4999 questions generated
OR: 4999 questions generated
NEITHER: 4999 questions generated
Mixed: 4999 questions generated

AND QA saved: 4999 questions in QA_AND.csv
  Label distribution: {'A': 1250, 'B': 1250, 'C': 1250, 'D': 1249}
  Label sequence (first 8): ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

OR QA saved: 4999 questions in QA_OR.csv
  Label distribution: {'A': 1250, 'B': 1250, 'C': 1250, 'D': 1249}
  Label sequence (first 8): ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

NEITHER QA saved: 4999 questions in QA_NEITHER.csv
  Label distribution: {'A': 1250, 'B': 1250, 'C': 1250, 'D': 1249}
  Label sequence (first 8): ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

Mixed QA saved: 4999 questions in QA_Mixed.csv
  Label distribution: {'A': 1250, 'B': 1250, 'C': 1250, 'D': 1249}
  Label sequence (first 8): ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D']

Final total questions generated: 19996


In [7]:
qa_df = pd.read_csv('/kaggle/working/QA_Mixed.csv')
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text,qa_type
0,The sanctions against the school were a punish...,undermine OR accelerate change,enforce rules OR facilitate growth,NEITHER negate NOR facilitate growth,NEITHER undermine NOR facilitate growth,A,undermine OR accelerate change,Mixed
1,Sammy wanted to go to where the people were. ...,NEITHER local events NOR quiet retreats,community gatherings OR sports arenas,train platforms AND empty parks,community gatherings AND sports arenas,B,community gatherings OR sports arenas,Mixed
2,To locate a choker not located in a jewelry bo...,fashion outlet AND personal accessories,NEITHER online marketplace NOR personal access...,artisan market AND vintage fair,artisan market AND home decor store,C,artisan market AND vintage fair,Mixed
3,Google Maps and other highway and street GPS s...,historical navigation logs AND tourist informa...,NEITHER printed road maps NOR route planning s...,manual navigation techniques AND tourist infor...,manual navigation techniques AND route plannin...,D,manual navigation techniques AND route plannin...,Mixed
4,"The fox walked from the city into the forest, ...",natural habitat AND shelter from disturbances,farm fields OR neighborhood pets,city park AND neighborhood pets,farm fields AND neighborhood pets,A,natural habitat AND shelter from disturbances,Mixed


In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_qa_metrics(results_df):
    """
    Calculate comprehensive evaluation metrics for QA results
    
    Args:
        results_df: DataFrame with 'predicted_label' and 'correct_label' columns
    
    Returns:
        dict: Dictionary containing all evaluation metrics
    """
    
    # Remove None predictions for accurate calculation
    valid_predictions = results_df.dropna(subset=['predicted_label'])
    
    if len(valid_predictions) == 0:
        print("Warning: No valid predictions found!")
        return {}
    
    y_true = valid_predictions['correct_label'].tolist()
    y_pred = valid_predictions['predicted_label'].tolist()
    
    # Basic metrics
    total_questions = len(results_df)
    answered_questions = len(valid_predictions)
    unanswered_questions = total_questions - answered_questions
    
    # Accuracy (main metric for QA)
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    labels = ['A', 'B', 'C', 'D']
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    
    # Macro and micro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Answer rate (percentage of questions answered)
    answer_rate = answered_questions / total_questions
    
    # Compile results
    metrics = {
        'total_questions': total_questions,
        'answered_questions': answered_questions,
        'unanswered_questions': unanswered_questions,
        'answer_rate': answer_rate,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
    }
    
    # Per-class metrics
    for i, label in enumerate(labels):
        metrics[f'{label}_precision'] = precision[i]
        metrics[f'{label}_recall'] = recall[i]
        metrics[f'{label}_f1'] = f1[i]
        metrics[f'{label}_support'] = support[i]
    
    return metrics, cm, y_true, y_pred

def print_qa_metrics(metrics):
    """Pretty print the evaluation metrics"""
    print("=" * 50)
    print("QA EVALUATION METRICS")
    print("=" * 50)
    
    print(f"Total Questions: {metrics['total_questions']}")
    print(f"Answered Questions: {metrics['answered_questions']}")
    print(f"Answer Rate: {metrics['answer_rate']:.1%}")
    print(f"Accuracy: {metrics['accuracy']:.1%}")
    
    print("\n" + "-" * 30)
    print("OVERALL METRICS")
    print("-" * 30)
    print(f"Macro Precision: {metrics['macro_precision']:.3f}")
    print(f"Macro Recall: {metrics['macro_recall']:.3f}")
    print(f"Macro F1: {metrics['macro_f1']:.3f}")
    print(f"Micro Precision: {metrics['micro_precision']:.3f}")
    print(f"Micro Recall: {metrics['micro_recall']:.3f}")
    print(f"Micro F1: {metrics['micro_f1']:.3f}")
    
    print("\n" + "-" * 40)
    print("PER-CLASS METRICS")
    print("-" * 40)
    print(f"{'Class':<5} {'Precision':<9} {'Recall':<6} {'F1':<6} {'Support'}")
    print("-" * 40)
    
    for label in ['A', 'B', 'C', 'D']:
        precision = metrics[f'{label}_precision']
        recall = metrics[f'{label}_recall']
        f1 = metrics[f'{label}_f1']
        support = int(metrics[f'{label}_support'])
        print(f"{label:<5} {precision:<9.3f} {recall:<6.3f} {f1:<6.3f} {support}")

def plot_confusion_matrix(cm, labels=['A', 'B', 'C', 'D'], save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_answer_distribution(results_df):
    """Analyze the distribution of correct and predicted answers"""
    print("\n" + "=" * 50)
    print("ANSWER DISTRIBUTION ANALYSIS")
    print("=" * 50)
    
    # True label distribution
    true_dist = results_df['correct_label'].value_counts().sort_index()
    print("\nTrue Label Distribution:")
    for label in ['A', 'B', 'C', 'D']:
        count = true_dist.get(label, 0)
        pct = count / len(results_df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Predicted label distribution (excluding None)
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        pred_dist = valid_preds['predicted_label'].value_counts().sort_index()
        print("\nPredicted Label Distribution:")
        for label in ['A', 'B', 'C', 'D']:
            count = pred_dist.get(label, 0)
            pct = count / len(valid_preds) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Check for bias towards certain options
    print("\nLabel Rotation Check:")
    if len(set(true_dist.values)) == 1:
        print("✓ Perfect label rotation detected")
    else:
        print("⚠ Uneven label distribution detected")
        
def comprehensive_qa_evaluation(results_df, save_plots=True):
    """Run complete evaluation pipeline"""
    
    # Calculate metrics
    metrics, cm, y_true, y_pred = calculate_qa_metrics(results_df)
    
    if not metrics:
        return
    
    # Print metrics
    print_qa_metrics(metrics)
    
    # Analyze distributions
    analyze_answer_distribution(results_df)
    
    # Plot confusion matrix
    if save_plots:
        plot_confusion_matrix(cm, save_path='confusion_matrix.png')
    else:
        plot_confusion_matrix(cm)
    
    # Print sklearn classification report for additional insights
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        print("\n" + "=" * 50)
        print("DETAILED CLASSIFICATION REPORT")
        print("=" * 50)
        print(classification_report(
            valid_preds['correct_label'], 
            valid_preds['predicted_label'],
            labels=['A', 'B', 'C', 'D']
        ))
    
    return metrics

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")

2025-09-08 22:40:52.858089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757371253.057086      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757371253.115012      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [10]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # More explicit prompt for single character output
            prompt = f"""Question: {str(row['question'])}
A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}
Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
Answer:"""

            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [11]:
qa_df = pd.read_csv('/kaggle/working/QA_AND.csv')
results = zero_shot_qa(model, tokenizer, qa_df)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_AND_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for op

({'total_questions': 4999, 'answered_questions': 4999, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.7053410682136427, 'macro_precision': 0.7082808316052108, 'macro_recall': 0.7053377101681345, 'macro_f1': 0.7050319840561663, 'micro_precision': 0.7053410682136427, 'micro_recall': 0.7053410682136427, 'micro_f1': 0.7053410682136427, 'A_precision': 0.7075757575757575, 'A_recall': 0.7472, 'A_f1': 0.7268482490272373, 'A_support': 1250, 'B_precision': 0.765490943755958, 'B_recall': 0.6424, 'B_f1': 0.6985645933014354, 'B_support': 1250, 'C_precision': 0.6871301775147929, 'C_recall': 0.7432, 'C_f1': 0.7140661029976941, 'C_support': 1250, 'D_precision': 0.672926447574335, 'D_recall': 0.688550840672538, 'D_f1': 0.6806489908982984, 'D_support': 1249}, array([[934,  59, 124, 133],
       [136, 803, 132, 179],
       [123,  92, 929, 106],
       [127,  95, 167, 860]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 

In [12]:
qa_df = pd.read_csv('/kaggle/working/QA_OR.csv')
results = zero_shot_qa(model, tokenizer, qa_df)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_OR_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 4999, 'answered_questions': 4999, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.6427285457091418, 'macro_precision': 0.6520317280511311, 'macro_recall': 0.6427141713370697, 'macro_f1': 0.6426318042158063, 'micro_precision': 0.6427285457091418, 'micro_recall': 0.6427285457091418, 'micro_f1': 0.6427285457091418, 'A_precision': 0.5949685534591195, 'A_recall': 0.7568, 'A_f1': 0.6661971830985915, 'A_support': 1250, 'B_precision': 0.7720883534136547, 'B_recall': 0.6152, 'B_f1': 0.684772929652716, 'B_support': 1250, 'C_precision': 0.642915642915643, 'C_recall': 0.628, 'C_f1': 0.6353702954269527, 'C_support': 1250, 'D_precision': 0.5981543624161074, 'D_recall': 0.5708566853482786, 'D_f1': 0.5841868086849652, 'D_support': 1249}, array([[946,  50, 111, 143],
       [198, 769, 131, 152],
       [204,  77, 785, 184],
       [242, 100, 194, 713]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 

In [13]:
qa_df = pd.read_csv('/kaggle/working/QA_NEITHER.csv')
results = zero_shot_qa(model, tokenizer, qa_df)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_NEITHER_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 4999, 'answered_questions': 4999, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.1194238847769554, 'macro_precision': 0.11391862301512398, 'macro_recall': 0.1194248198558847, 'macro_f1': 0.11374928241482223, 'micro_precision': 0.1194238847769554, 'micro_recall': 0.1194238847769554, 'micro_f1': 0.1194238847769554, 'A_precision': 0.1912640635340834, 'A_recall': 0.2312, 'A_f1': 0.2093444404201376, 'A_support': 1250, 'B_precision': 0.08456973293768547, 'B_recall': 0.0456, 'B_f1': 0.059251559251559255, 'B_support': 1250, 'C_precision': 0.09266409266409266, 'C_recall': 0.0768, 'C_f1': 0.08398950131233594, 'C_support': 1250, 'D_precision': 0.08717660292463442, 'D_recall': 0.12409927942353884, 'D_f1': 0.10241162867525604, 'D_support': 1249}, array([[289, 211, 272, 478],
       [366,  57, 306, 521],
       [364, 166,  96, 624],
       [492, 240, 362, 155]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A

In [14]:
qa_df = pd.read_csv('/kaggle/working/QA_Mixed.csv')
results = zero_shot_qa(model, tokenizer, qa_df)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_MIXED_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 4999, 'answered_questions': 4999, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.4728945789157832, 'macro_precision': 0.47413180275302835, 'macro_recall': 0.47289975980784626, 'macro_f1': 0.4723104896625971, 'micro_precision': 0.4728945789157832, 'micro_recall': 0.4728945789157832, 'micro_f1': 0.4728945789157832, 'A_precision': 0.47275641025641024, 'A_recall': 0.472, 'A_f1': 0.47237790232185745, 'A_support': 1250, 'B_precision': 0.49524714828897337, 'B_recall': 0.4168, 'B_f1': 0.4526498696785404, 'B_support': 1250, 'C_precision': 0.46255506607929514, 'C_recall': 0.504, 'C_f1': 0.4823889739663093, 'C_support': 1250, 'D_precision': 0.46596858638743455, 'D_recall': 0.49879903923138513, 'D_f1': 0.48182521268368134, 'D_support': 1249}, array([[590, 195, 217, 248],
       [225, 521, 264, 240],
       [212, 182, 630, 226],
       [221, 154, 251, 623]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 